In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import math
import random

In [17]:
nu = 5e-4            
rho = 1
U_inlet = 1.0

cyl_center = (0.5, 0.5)
cyl_radius = 0.05

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:
data_dir = "/kaggle/input/cfd-flow-pass-a-cylinder-0-01"
t_start = 0
t_end = 501
dt = 0.01

xyt_list = []
uvp_list = []

for i in range(t_start, t_end):
    csv_path = os.path.join(data_dir, f"result_{i}.csv")
    df = pd.read_csv(csv_path)

    # timestep
    t_val = i * dt
    t_column = np.full_like(df["Points:0"].values, fill_value=t_val, dtype=np.float32)

    # (x, y, t)
    xyt = np.stack([
        df["Points:0"].values,
        df["Points:1"].values,
        t_column
    ], axis=1)  

    # (u, v, p)
    uvp = np.stack([
        df["u:0"].values,
        df["u:1"].values,
        df["p"].values
    ], axis=1)  

    xyt_list.append(xyt)
    uvp_list.append(uvp)

xyt_tensor = torch.tensor(np.concatenate(xyt_list, axis=0), dtype=torch.float32)
uvp_tensor = torch.tensor(np.concatenate(uvp_list, axis=0), dtype=torch.float32)

print(xyt_tensor.shape)
print(uvp_tensor.shape)

torch.Size([20806530, 3])
torch.Size([20806530, 3])


In [22]:
class ResNetPINN(nn.Module):
    def __init__(self, layers=[3] + [128]*20 + [3]):
        super().__init__()
        self.input_layer = nn.Linear(layers[0], layers[1])
        self.hidden_layers = nn.ModuleList()
        for i in range(1, len(layers)-2):
            self.hidden_layers.append(nn.Linear(layers[i], layers[i+1]))
        self.output_layer = nn.Linear(layers[-2], layers[-1])
        self.activation = nn.Tanh()

        # Xavier initialization
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        # Input layer
        h = self.activation(self.input_layer(x))

        # Residual blocks
        for layer in self.hidden_layers:
            h_in = h
            h = self.activation(layer(h))
            # Add residual connection (only if dimensions match)
            if h.shape == h_in.shape:
                h = h + h_in

        # Output layer (no activation)
        out = self.output_layer(h)
        return out  # [u, v, p]

In [23]:
model = ResNetPINN().to(device)

In [24]:
model.load_state_dict(torch.load("/kaggle/input/resnet-model/resnet_pinn_ns_adam_final.pth"))

<All keys matched successfully>

In [25]:
model.eval()

ResNetPINN(
  (input_layer): Linear(in_features=3, out_features=128, bias=True)
  (hidden_layers): ModuleList(
    (0-18): 19 x Linear(in_features=128, out_features=128, bias=True)
  )
  (output_layer): Linear(in_features=128, out_features=3, bias=True)
  (activation): Tanh()
)

In [26]:
import torch
import numpy as np
from scipy.interpolate import griddata
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColorBar, LinearColorMapper, BasicTicker, PrintfTickFormatter
from bokeh.palettes import Turbo256

output_notebook()

# --- SETTINGS ---
target_t = 0.5
tol = 1e-5
grid_x, grid_y = 250, 100  # maintain 2.5:1 ratio (e.g. 250x100 grid)

# --- 1️⃣ Filter timestep ---
mask = torch.isclose(
    xyt_tensor[:, 2],
    torch.tensor(target_t, device=xyt_tensor.device),
    atol=tol
)
xyt_slice = xyt_tensor[mask]
uvp_slice = uvp_tensor[mask]

print(f"Selected {len(xyt_slice)} points for t={target_t}")

# --- 2️⃣ Predict ---
with torch.no_grad():
    pred = model(xyt_slice.to("cuda"))

# --- 3️⃣ Move to CPU ---
x = xyt_slice[:, 0].cpu().numpy()
y = xyt_slice[:, 1].cpu().numpy()
u_real = uvp_slice[:, 0].cpu().numpy()
v_real = uvp_slice[:, 1].cpu().numpy()
pressure_real = uvp_slice[:, 2].cpu().numpy()
u_pred = pred[:, 0].cpu().numpy()
v_pred = pred[:, 1].cpu().numpy()
pressure_pred = pred[:, 2].cpu().numpy()

# --- 4️⃣ Velocity magnitude ---
vel_real = np.sqrt(u_real**2 + v_real**2)
vel_pred = np.sqrt(u_pred**2 + v_pred**2)

# --- 5️⃣ Interpolate to 2.5:1 domain grid ---
x_lin = np.linspace(x.min(), x.max(), grid_x)
y_lin = np.linspace(y.min(), y.max(), grid_y)
X, Y = np.meshgrid(x_lin, y_lin)

def interp(Z):
    return griddata((x, y), Z, (X, Y), method="cubic")

U_real = interp(u_real)
U_pred = interp(u_pred)
V_real = interp(v_real)
V_pred = interp(v_pred)
Vel_real = interp(vel_real)
Vel_pred = interp(vel_pred)
Pressure_real = interp(pressure_real)
Pressure_pred = interp(pressure_pred)


# --- 6️⃣ Shared color mappers ---
def get_mapper(real, pred):
    low = min(np.nanmin(real), np.nanmin(pred))
    high = max(np.nanmax(real), np.nanmax(pred))
    return LinearColorMapper(palette=Turbo256, low=low, high=high)

mapper_u = get_mapper(U_real, U_pred)
mapper_v = get_mapper(V_real, V_pred)
mapper_vel = get_mapper(Vel_real, Vel_pred)
mapper_pressure = get_mapper(Pressure_real, Pressure_pred)

# --- 7️⃣ Plot helper (enforce same physical size & domain limits) ---
def make_plot(X, Y, Z, title, mapper):
    p = figure(
        title=title,
        width=500, height=200,  # maintain 2.5:1 visually
        x_range=(X.min(), X.max()),
        y_range=(Y.min(), Y.max()),
        x_axis_label="x", y_axis_label="y",
        match_aspect=True,
        tools="pan,wheel_zoom,reset,save"
    )
    p.image(image=[Z], x=X.min(), y=Y.min(),
            dw=X.max()-X.min(), dh=Y.max()-Y.min(),
            color_mapper=mapper)
    return p

# --- 8️⃣ Add colorbar helper ---
def add_colorbar(p, mapper, label):
    color_bar = ColorBar(
        color_mapper=mapper, ticker=BasicTicker(),
        formatter=PrintfTickFormatter(format="%.3f"),
        label_standoff=10, location=(0, 0),
        title=label, width=10
    )
    p.add_layout(color_bar, "right")

# --- 9️⃣ Make plots ---
p_u_real = make_plot(X, Y, U_real, f"Real u:0 (t={target_t})", mapper_u)
p_u_pred = make_plot(X, Y, U_pred, f"Predicted u:0 (t={target_t})", mapper_u)
add_colorbar(p_u_pred, mapper_u, "u:0")

p_v_real = make_plot(X, Y, V_real, f"Real u:1 (t={target_t})", mapper_v)
p_v_pred = make_plot(X, Y, V_pred, f"Predicted u:1 (t={target_t})", mapper_v)
add_colorbar(p_v_pred, mapper_v, "u:1")

p_vel_real = make_plot(X, Y, Vel_real, f"Real |U| (t={target_t})", mapper_vel)
p_vel_pred = make_plot(X, Y, Vel_pred, f"Predicted |U| (t={target_t})", mapper_vel)
add_colorbar(p_vel_pred, mapper_vel, "|U|")

p_pressure_real = make_plot(X, Y, Pressure_real, f"Real p (t={target_t})", mapper_pressure)
p_pressure_pred = make_plot(X, Y, Pressure_pred, f"Predicted p (t={target_t})", mapper_pressure)
add_colorbar(p_pressure_pred, mapper_pressure, "p")

# --- 🔟 Layout: 3 rows, 2 columns ---
layout = column(
    row(p_u_real, p_u_pred),
    row(p_v_real, p_v_pred),
    row(p_vel_real, p_vel_pred),
    row(p_pressure_real, p_pressure_pred)
)

show(layout)


Loading BokehJS ...

Selected 41530 points for t=0.5


In [27]:
import torch
import numpy as np
from scipy.interpolate import griddata
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColorBar, LinearColorMapper, BasicTicker, PrintfTickFormatter
from bokeh.palettes import Turbo256
from bokeh.io import export_png

output_notebook()

# --- SETTINGS ---
target_t = i*0.01
tol = 1e-5
grid_x, grid_y = 250, 100  # maintain 2.5:1 ratio (e.g. 250x100 grid)

# --- 1️⃣ Filter timestep ---
mask = torch.isclose(
    xyt_tensor[:, 2],
    torch.tensor(target_t, device=xyt_tensor.device),
    atol=tol
)
xyt_slice = xyt_tensor[mask]
uvp_slice = uvp_tensor[mask]

print(f"Selected {len(xyt_slice)} points for t={target_t}")

# --- 2️⃣ Predict ---
with torch.no_grad():
    pred = model(xyt_slice.to("cuda"))

# --- 3️⃣ Move to CPU ---
x = xyt_slice[:, 0].cpu().numpy()
y = xyt_slice[:, 1].cpu().numpy()
u_real = uvp_slice[:, 0].cpu().numpy()
v_real = uvp_slice[:, 1].cpu().numpy()
u_pred = pred[:, 0].cpu().numpy()
v_pred = pred[:, 1].cpu().numpy()

# --- 4️⃣ Velocity magnitude ---
vel_real = np.sqrt(u_real**2 + v_real**2)
vel_pred = np.sqrt(u_pred**2 + v_pred**2)

# --- 5️⃣ Interpolate to 2.5:1 domain grid ---
x_lin = np.linspace(x.min(), x.max(), grid_x)
y_lin = np.linspace(y.min(), y.max(), grid_y)
X, Y = np.meshgrid(x_lin, y_lin)

def interp(Z):
    return griddata((x, y), Z, (X, Y), method="cubic")

U_real = interp(u_real)
U_pred = interp(u_pred)
V_real = interp(v_real)
V_pred = interp(v_pred)
Vel_real = interp(vel_real)
Vel_pred = interp(vel_pred)

# --- 6️⃣ Shared color mappers ---
def get_mapper(real, pred):
    low = min(np.nanmin(real), np.nanmin(pred))
    high = max(np.nanmax(real), np.nanmax(pred))
    return LinearColorMapper(palette=Turbo256, low=low, high=high)

mapper_u = get_mapper(U_real, U_pred)
mapper_v = get_mapper(V_real, V_pred)
mapper_vel = get_mapper(Vel_real, Vel_pred)

# --- 7️⃣ Plot helper (enforce same physical size & domain limits) ---
def make_plot(X, Y, Z, title, mapper):
    p = figure(
        title=title,
        width=500, height=200,  # maintain 2.5:1 visually
        x_range=(X.min(), X.max()),
        y_range=(Y.min(), Y.max()),
        x_axis_label="x", y_axis_label="y",
        match_aspect=True,
        tools="pan,wheel_zoom,reset,save"
    )
    p.image(image=[Z], x=X.min(), y=Y.min(),
            dw=X.max()-X.min(), dh=Y.max()-Y.min(),
            color_mapper=mapper)
    return p

# --- 8️⃣ Add colorbar helper ---
def add_colorbar(p, mapper, label):
    color_bar = ColorBar(
        color_mapper=mapper, ticker=BasicTicker(),
        formatter=PrintfTickFormatter(format="%.3f"),
        label_standoff=10, location=(0, 0),
        title=label, width=10
    )
    p.add_layout(color_bar, "right")

# --- 9️⃣ Make plots ---
p_u_real = make_plot(X, Y, U_real, f"Real u:0 (t={target_t})", mapper_u)
p_u_pred = make_plot(X, Y, U_pred, f"Predicted u:0 (t={target_t})", mapper_u)
add_colorbar(p_u_pred, mapper_u, "u:0")

p_v_real = make_plot(X, Y, V_real, f"Real u:1 (t={target_t})", mapper_v)
p_v_pred = make_plot(X, Y, V_pred, f"Predicted u:1 (t={target_t})", mapper_v)
add_colorbar(p_v_pred, mapper_v, "u:1")

p_vel_real = make_plot(X, Y, Vel_real, f"Real |U| (t={target_t})", mapper_vel)
p_vel_pred = make_plot(X, Y, Vel_pred, f"Predicted |U| (t={target_t})", mapper_vel)
add_colorbar(p_vel_pred, mapper_vel, "|U|")

# --- 🔟 Layout: 3 rows, 2 columns ---
layout = column(
    row(p_u_real, p_u_pred),
    row(p_v_real, p_v_pred),
    row(p_vel_real, p_vel_pred)
)

show(layout)


Loading BokehJS ...

Selected 41530 points for t=5.0
